# Intel Stock Prediction using LSTM Keras

<img src="https://images.pexels.com/photos/159888/pexels-photo-159888.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940" width="800px">

#### Here, we are going to use Intel Stock Dataset from Kaggle to use it for our lstm Keras model for prediction

In [ ]:

import numpy as np 
import pandas as pd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data_s = pd.read_csv('/kaggle/input/intel-stock-prices-historical-data-intc/INTC.csv')

data_s

Extracting Date, Open, High, Low and Close from the Dataframe.

In [ ]:
data = data_s[['Date', 'Open', 'High', 'Low', 'Close']]

Using Plotly we will see how is the stock looks like.

In [ ]:

import plotly.graph_objs as go

fig=go.Figure(data=[go.Scatter(name = 'Close',x= data['Date'],y= data['Close']),
                   go.Scatter(name = 'Open',x= data['Date'],y= data['Open']),
                   go.Scatter(name = 'High',x= data['Date'],y= data['High']),
                   go.Scatter(name = 'Low',x= data['Date'],y= data['Low'])
                    ])

fig.show()

From the plot stock data from year 1995 onwards will be used.

In [ ]:
dfs = data.loc[data['Date'] >'1995']

#dfs = data

dfs

Extracing High, Low, Close from the previous dataframe and load to other and discarding date.

In [ ]:
df = dfs.iloc[:,1:]

df

Below seaborn libray is used to see correlation between the features in HeatMap visualization.

In [ ]:
import seaborn as sns

sns.heatmap(df.corr(), annot=True, cmap='RdYlGn', linewidths=0.1, vmin=0)

For the LSTM Model Splitting the dataframe to make training dataset.

In [ ]:
training_set = df.iloc[:round(len(df)*0.8)].values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
training_set = scaler.fit_transform(training_set)

In [ ]:
print(training_set.shape)

Here, we are going to make such arrangement with the training dataset that previous 28 Days will be used to predict 29th day.

In [ ]:
trainX = [] 
trainY = []

in_future = 1 # Number of days we want to predict into the future 
in_past = 28 # Number of past days we want to use to predict the future

for i in range(in_past, len(training_set) - in_future +1): 
    trainX.append(training_set[i - in_past:i, 0:training_set.shape[1]]) 
    trainY.append(training_set[i + in_future - 1:i + in_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape = {}.'.format(trainX.shape)) 
print('trainY shape = {}.'.format(trainY.shape))

Preparing Keras Model.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(trainX.shape[0], trainX.shape[2]), return_sequences=True))
model.add(LSTM(25, activation='relu', return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(4))

model.compile(optimizer='adam', loss='mse')
model.summary()

Fitting the Model to the training dataset

In [ ]:
history = model.fit(trainX, trainY, epochs=30, batch_size=50, validation_split=0.2, verbose=1)

Ploting the Loss and valid loss so as to observe the learning of the Model.

In [ ]:
import matplotlib.pyplot as plt


plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend();

It seems that the model is learning quite well. 
And the overfitting is not present in our model.

Now making dataset for our model to predict.

In [ ]:
inputs_data =df.iloc[round(len(df)*0.8):,:].values

scaler = MinMaxScaler(feature_range=(0, 1))
inputs_data = scaler.fit_transform(inputs_data)


prediction_stocks = []

for i in range(28, len(inputs_data)):
    prediction_stocks.append(inputs_data[i-28:i])
    
prediction_stocks = np.array(prediction_stocks)

Now we are going to feed in the prediction dataset to the model.

In [ ]:
predictions = model.predict(prediction_stocks)

### The predicted data need's to be invert transformed so that the values returns into the original scale.

In [ ]:

#forecast_copies = np.repeat(predictions, prediction_stocks.shape[2], axis=-1)
y_pred_future = scaler.inverse_transform(predictions)

### Taking these predicted data and making it into dataframe so that it can be interpret.

In [ ]:
df_forecast = pd.DataFrame({'Date':dfs.Date[-y_pred_future.shape[0]:],
                            'Open': y_pred_future[:,0],
                            'High':y_pred_future[:,1],
                            'Low':y_pred_future[:,2], 
                            'Close':y_pred_future[:,3]})

df_forecast

 ### Ploting the predicted 'Close' Stock Price with Original 'Close' Stock Price.

In [ ]:
import plotly.graph_objs as go

fig2=go.Figure(data=[go.Scatter(name = 'Close',x= dfs['Date'],y= dfs['Close']),
                   go.Scatter(name = 'Close_Predicted',x= df_forecast['Date'],y= df_forecast['Close'])
                    ])

fig2.show()

#### Looks like the Prediction is quite well

### Calculation of prediction interval(90% confidence level)

For 90% confidence level, t-multiplier is 1.645 which is calculated from degree of freedom of the sample.


t-multiplier(1.645) * standard deviation(stdev) gives the magnitude of interval.

The minimum and maximum range is given by :
confidence min = value-interval
confidence max = value + interval

In [ ]:
sum_err = np.sum((dfs.Close[-y_pred_future.shape[0]:]-df_forecast['Close'])**2)

print(sum_err)

x = (1/(len(df_forecast))*sum_err)

stdev = np.sqrt(x)

interval = 	1.645*stdev

print(interval)

#### Adding the Close Confidence Min and Close Confidence Max to the dataframe, Plus the original closing price data too.

In [ ]:
df_forecast['Close Confidence Min'] = df_forecast['Close'] - interval

df_forecast['Close Confidence Max'] = df_forecast['Close'] + interval

df_forecast['Orginal Close'] = dfs.Close[-y_pred_future.shape[0]:]

df_forecast

### Ploting the Closing Price(Original and Predicted) with the Close Confidence Min as 'Lower Bound' and Close Confidence Max as 'Upper Bound' which gives the idea of the Model's 90% Confidence Level.

In [ ]:

     
fig3=go.Figure(data=[
                        go.Scatter(
                                    name = 'Close',
                                    x= df_forecast['Date'],
                                    y= df_forecast['Orginal Close']),
                     
                        go.Scatter(
                                    name = 'Close Predicted',
                                    x= df_forecast['Date'],
                                    y= df_forecast['Close']),

                        go.Scatter(
                                    name='Upper Bound',
                                    x= df_forecast['Date'],
                                    y= df_forecast['Close Confidence Max'],
                                    mode='lines',
                                    marker=dict(color="green"),
                                    line=dict(width=1),
                                    showlegend=False),

                         go.Scatter(
                                    name='Lower Bound',
                                    x= df_forecast['Date'],
                                    y= df_forecast['Close Confidence Min'],
                                    marker=dict(color="purple"),
                                    line=dict(width=1),
                                    mode='lines',
                                    fillcolor='rgba(68, 68, 68, 0.3)',
                                    fill='tonexty',
                                    showlegend=False)

                    ])

fig3.show()

#### There are instances where the original Closing Price is out of bound from the 90% Confidence Level.

#### But in general sense the model tracks the original Closing Price inside the 90% Confidence Level Bound.